<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-06 13:36:54
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.38 C
-------------------
Today PnL: -1.04 L (-0.75%)
Current PnL: -31.59 L (-20.14%)
CY Booked + Current PnL: -16.85 L (-10.74%)
-------------------
Total profit:  1.02 L
Total loss:  -32.60 L
-------------------
Total Booked + Current PnL: 10.13 L (7.81%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.69%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 96.97 L (70.32%)
Deployed:  1.30 C
Current:  1.38 C
CAGR/XIRR %: 3.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.61,5.55,12.96,19.23,20581.0,8344.0,158804.0,484.83,24.27,53.0,L-LC,12.41,182.0,0.41,1.15,28.00,XY25,ATH,MINING
50,MASFIN,2.03,-2.05,24.11,21.57,23138.0,-2010.0,95970.0,397.04,-15.54,54.0,H-SC,3.24,168.0,-0.09,0.70,40.05,XR,ATH,FINANCE
77,TTKPRESTIG,1.22,-26.72,36.59,0.09,27022.0,-26926.0,73851.0,770.00,71.40,38.0,M-SC,8.22,253.0,-1.00,0.54,1.64,OX40N,NTT,DURABLES
51,MEDANTA,2.46,-5.23,30.93,24.08,37212.0,-6640.0,120310.0,1486.00,-12.87,52.0,X-SC,11.31,91.0,-0.18,0.87,10.19,XY24,NTT,HEALTHCARE
57,RECLTD,-1.96,0.19,20.51,20.74,41749.0,385.0,203555.0,446.00,31.15,54.0,M-MC,4.02,189.0,0.01,1.48,12.18,XY25,NTT,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,4.74,-20.37,39.12,10.78,74862.0,-48952.0,191366.0,452.00,-52.60,45.0,X-LC,18.72,1.0,-0.65,1.39,5.40,X40,NTT,FMCG
26,FINCABLES,3.00,-2.74,113.77,107.91,163370.0,-4047.0,143597.0,1641.55,-20.96,58.0,M-SC,11.55,220.0,-0.02,1.04,8.03,OX40N,ATH,CABLES
51,MEDANTA,2.46,-5.23,30.93,24.08,37212.0,-6640.0,120310.0,1486.00,-12.87,52.0,X-SC,11.31,91.0,-0.18,0.87,10.19,XY24,NTT,HEALTHCARE
50,MASFIN,2.03,-2.05,24.11,21.57,23138.0,-2010.0,95970.0,397.04,-15.54,54.0,H-SC,3.24,168.0,-0.09,0.70,40.05,XR,ATH,FINANCE
15,CAMPUS,1.86,-7.06,43.40,33.28,67675.0,-11841.0,155934.0,393.00,-25.19,63.0,M-SC,5.18,221.0,-0.17,1.13,20.86,XY24,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJESHEXPO,-5.00,-66.87,202.01,0.07,91820.0,-91724.0,45453.0,518.0,1565.40,49.0,L-SC,11.42,269.0,-1.00,0.33,13.04,OX40N,NTT,JEWELLERY
14,BSOFT,-4.80,-20.51,89.69,50.79,100278.0,-28848.0,111805.0,831.7,2.05,55.0,H-SC,6.84,171.0,-0.29,0.81,30.18,XR,ATH,IT
72,TATAELXSI,-4.74,-25.78,77.88,32.03,76206.0,-33985.0,97850.0,9161.0,-19.73,39.0,H-SC,6.38,157.0,-0.45,0.71,9.60,OX40N,NTT,IT
38,IEX,-3.76,-18.46,82.27,48.63,134136.0,-36910.0,163044.0,219.0,-45.72,36.0,H-SC,16.68,137.0,-0.28,1.18,0.00,XR,NTT,MISC
63,SFL,-3.73,-42.17,118.95,26.63,180390.0,-110566.0,151652.0,1287.0,14.87,58.0,M-SC,16.08,241.0,-0.61,1.10,16.41,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,-1.96,0.19,20.51,20.74,41749.0,385.0,203555.0,446.0,31.15,54.0,M-MC,4.02,189.0,0.01,1.48,12.18,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,3.00,-2.74,113.77,107.91,163370.0,-4047.0,143597.0,1641.55,-20.96,58.0,M-SC,11.55,220.0,-0.02,1.04,8.03,OX40N,ATH,CABLES
66,SIS,-0.79,-21.68,54.84,21.26,47686.0,-24077.0,86955.0,528.00,2054.92,63.0,H-SC,4.57,163.0,-0.50,0.63,19.88,OX40N,NTT,MISC
72,TATAELXSI,-4.74,-25.78,77.88,32.03,76206.0,-33985.0,97850.0,9161.00,-19.73,39.0,H-SC,6.38,157.0,-0.45,0.71,9.60,OX40N,NTT,IT
77,TTKPRESTIG,1.22,-26.72,36.59,0.09,27022.0,-26926.0,73851.0,770.00,71.40,38.0,M-SC,8.22,253.0,-1.00,0.54,1.64,OX40N,NTT,DURABLES
40,INDIGOPNTS,0.23,-27.98,38.88,0.02,48876.0,-48848.0,125711.0,1408.00,91.96,29.0,M-SC,5.54,240.0,-1.00,0.91,9.35,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,-1.96,0.19,20.51,20.74,41749.0,385.0,203555.0,446.00,31.15,54.0,M-MC,4.02,189.0,0.01,1.48,12.18,XY25,NTT,FINANCE
18,COALINDIA,-0.61,5.55,12.96,19.23,20581.0,8344.0,158804.0,484.83,24.27,53.0,L-LC,12.41,182.0,0.41,1.15,28.00,XY25,ATH,MINING
1,ABB,0.31,6.19,37.08,45.57,103017.0,16205.0,277824.0,7934.00,-31.79,72.0,H-MC,4.48,121.0,0.16,2.02,23.37,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.31,6.19,37.08,45.57,103017.0,16205.0,277824.0,7934.00,-31.79,72.0,H-MC,4.48,121.0,0.16,2.02,23.37,AR,NTT,ELECTRICAL
26,FINCABLES,3.00,-2.74,113.77,107.91,163370.0,-4047.0,143597.0,1641.55,-20.96,58.0,M-SC,11.55,220.0,-0.02,1.04,8.03,OX40N,ATH,CABLES
39,INDIAMART,-3.02,-4.86,117.29,106.72,137625.0,-5999.0,117337.0,4810.62,-52.92,49.0,H-SC,8.38,138.0,-0.04,0.85,17.38,AR,ATH,MISC
84,WIPRO,-1.37,-5.58,82.45,72.27,131911.0,-9459.0,159989.0,420.00,-18.78,36.0,M-LC,3.16,101.0,-0.07,1.16,2.84,XR,NTT,IT
50,MASFIN,2.03,-2.05,24.11,21.57,23138.0,-2010.0,95970.0,397.04,-15.54,54.0,H-SC,3.24,168.0,-0.09,0.70,40.05,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.36,-45.27,147.02,35.20,103540.0,-58244.0,70426.0,2096.00,-7.73,24.0,X-SC,21.12,93.0,-0.56,0.51,0.08,X40,NTT,FOOTWEAR
6,ASIANPAINT,-1.61,-16.36,44.60,20.95,93920.0,-41184.0,210584.0,3460.25,-17.22,27.0,X-LC,9.15,36.0,-0.44,1.53,13.45,X40,ATH,PAINTS
81,VBL,0.01,-11.70,53.64,35.67,149603.0,-36940.0,278902.0,671.64,-22.18,27.0,X-LC,0.71,5.0,-0.25,2.02,0.54,X40N,ATH,FMCG
13,BERGEPAINT,-0.91,-16.63,45.30,21.14,85862.0,-37799.0,189540.0,680.00,-22.51,30.0,X-MC,8.77,75.0,-0.44,1.37,1.10,XY24,NTT,PAINTS
42,INFY,-1.37,2.07,31.52,34.24,100666.0,6484.0,319372.0,1972.00,-20.67,30.0,X-LC,6.06,6.0,0.06,2.32,10.53,X40,NTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,0.01,-11.70,53.64,35.67,149603.0,-36940.0,278902.0,671.64,-22.18,27.0,X-LC,0.71,5.0,-0.25,2.02,0.54,X40N,ATH,FMCG
73,TCS,-2.09,-19.46,47.20,18.55,143779.0,-73621.0,304616.0,4311.58,-30.74,34.0,X-LC,0.71,3.0,-0.51,2.21,3.63,X40,ATH,IT
5,ANGELONE,-1.87,6.29,16.85,24.20,54670.0,19200.0,324450.0,3033.00,51.36,53.0,X-SC,1.53,99.0,0.35,2.35,35.88,X40N,NTT,FINANCE
21,DABUR,0.23,-1.18,45.41,43.70,111319.0,-2925.0,245143.0,735.00,-9.24,46.0,X-MC,2.05,73.0,-0.03,1.78,13.69,XY24,BTT,FMCG
34,HINDUNILVR,1.24,-7.27,22.58,13.67,53824.0,-18693.0,238370.0,2922.00,-14.28,53.0,X-LC,2.44,9.0,-0.35,1.73,12.53,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.36,-45.27,147.02,35.20,103540.0,-58244.0,70426.0,2096.00,-7.73,24.0,X-SC,21.12,93.0,-0.56,0.51,0.08,X40,NTT,FOOTWEAR
81,VBL,0.01,-11.70,53.64,35.67,149603.0,-36940.0,278902.0,671.64,-22.18,27.0,X-LC,0.71,5.0,-0.25,2.02,0.54,X40N,ATH,FMCG
13,BERGEPAINT,-0.91,-16.63,45.30,21.14,85862.0,-37799.0,189540.0,680.00,-22.51,30.0,X-MC,8.77,75.0,-0.44,1.37,1.10,XY24,NTT,PAINTS
8,AWL,-2.14,-31.88,130.74,57.19,290466.0,-103956.0,222171.0,485.00,-67.84,34.0,X-MC,16.59,58.0,-0.36,1.61,1.25,XY24,NTT,FMCG
3,ACC,-1.03,-30.28,135.44,64.15,224695.0,-72051.0,165900.0,3906.00,-59.37,40.0,X-SC,7.85,84.0,-0.32,1.20,1.41,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-2.36,-29.59,102.87,42.84,47085.0,-19235.0,45771.0,424.00,-53.72,55.0,X-SC,21.23,83.0,-0.41,0.33,10.10,XY24,NTT,MISC
12,BATAINDIA,-0.36,-45.27,147.02,35.20,103540.0,-58244.0,70426.0,2096.00,-7.73,24.0,X-SC,21.12,93.0,-0.56,0.51,0.08,X40,NTT,FOOTWEAR
58,RELAXO,-0.60,-52.03,217.49,52.31,151449.0,-75525.0,69635.0,1176.00,-48.89,41.0,X-SC,12.47,92.0,-0.50,0.51,3.55,X40N,NTT,FOOTWEAR
51,MEDANTA,2.46,-5.23,30.93,24.08,37212.0,-6640.0,120310.0,1486.00,-12.87,52.0,X-SC,11.31,91.0,-0.18,0.87,10.19,XY24,NTT,HEALTHCARE
35,HONAUT,-0.91,-20.91,81.21,43.32,104615.0,-34052.0,128820.0,58357.33,-32.00,43.0,X-SC,9.41,90.0,-0.33,0.93,4.60,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,4.74,-20.37,39.12,10.78,74862.0,-48952.0,191366.0,452.00,-52.60,45.0,X-LC,18.72,1.0,-0.65,1.39,5.40,X40,NTT,FMCG
75,TMPV,-1.64,-24.57,63.04,22.98,107410.0,-55514.0,170384.0,600.00,-21.86,58.0,X-LC,4.26,2.0,-0.52,1.24,8.92,XY24,NTT,AUTO
73,TCS,-2.09,-19.46,47.20,18.55,143779.0,-73621.0,304616.0,4311.58,-30.74,34.0,X-LC,0.71,3.0,-0.51,2.21,3.63,X40,ATH,IT
81,VBL,0.01,-11.70,53.64,35.67,149603.0,-36940.0,278902.0,671.64,-22.18,27.0,X-LC,0.71,5.0,-0.25,2.02,0.54,X40N,ATH,FMCG
42,INFY,-1.37,2.07,31.52,34.24,100666.0,6484.0,319372.0,1972.00,-20.67,30.0,X-LC,6.06,6.0,0.06,2.32,10.53,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-1.88,-33.76,100.61,32.89,51455.0,-26064.0,51143.0,1800.00,-731.43,49.0,L-MC,11.31,259.0,-0.51,0.37,40.84,XR,NTT,BANKS
67,SONACOMS,-2.84,-11.89,57.85,39.09,51565.0,-12023.0,89136.0,804.02,-31.03,60.0,M-SC,1.84,227.0,-0.23,0.65,26.10,AR,ATH,AUTO
14,BSOFT,-4.80,-20.51,89.69,50.79,100278.0,-28848.0,111805.0,831.70,2.05,55.0,H-SC,6.84,171.0,-0.29,0.81,30.18,XR,ATH,IT
18,COALINDIA,-0.61,5.55,12.96,19.23,20581.0,8344.0,158804.0,484.83,24.27,53.0,L-LC,12.41,182.0,0.41,1.15,28.00,XY25,ATH,MINING
1,ABB,0.31,6.19,37.08,45.57,103017.0,16205.0,277824.0,7934.00,-31.79,72.0,H-MC,4.48,121.0,0.16,2.02,23.37,AR,NTT,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.31,6.19,37.08,45.57,103017.0,16205.0,277824.0,7934.00,-31.79,72.0,H-MC,4.48,121.0,0.16,2.02,23.37,AR,NTT,ELECTRICAL
27,GILLETTE,1.19,3.10,25.82,29.72,67588.0,7869.0,261765.0,10978.86,-15.28,66.0,X-SC,11.24,87.0,0.12,1.90,21.13,X40,ATH,FMCG
5,ANGELONE,-1.87,6.29,16.85,24.20,54670.0,19200.0,324450.0,3033.00,51.36,53.0,X-SC,1.53,99.0,0.35,2.35,35.88,X40N,NTT,FINANCE
67,SONACOMS,-2.84,-11.89,57.85,39.09,51565.0,-12023.0,89136.0,804.02,-31.03,60.0,M-SC,1.84,227.0,-0.23,0.65,26.10,AR,ATH,AUTO
62,SATIN,0.11,-11.66,71.87,51.83,175415.0,-32212.0,244072.0,274.00,-21.16,65.0,H-SC,3.42,148.0,-0.18,1.77,18.90,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.32
1,25,44.68
2,50,76.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.32
MC    30.91
LC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.57
X40      23.52
X40N     15.01
AR        9.00
XY25      8.79
XR        8.77
OX40N     7.43
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.17
H-SC    22.87
X-LC    20.28
M-SC    11.39
X-SC    10.31
H-MC     4.89
M-MC     1.48
H-LC     1.17
M-LC     1.16
L-LC     1.15
L-SC     0.75
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.02,-14.31,52.72
FINANCE,13.50,-12.31,56.51
IT,12.26,-26.41,89.56
MISC,7.06,-33.16,87.63
ELECTRICAL,6.17,-10.23,50.45
PAINTS,5.21,-28.39,47.46
INSURANCE,4.75,-3.53,38.96
PHARMA,4.02,-5.97,39.33
AUTO,2.97,-28.44,72.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3305025.0,21
AR,1330217.0,10
XR,1282420.0,12
X40,1255347.0,15
X40N,1051343.0,10
OX40N,767814.0,10
XY25,408454.0,6
SR,296642.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3639823.0,24
X-MC,1695060.0,16
M-SC,1488622.0,15
X-LC,1096623.0,12
X-SC,868070.0,9
H-MC,404157.0,3
L-SC,187628.0,2
M-LC,131911.0,1
H-LC,71583.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1245385.0      6
           AR         942506.0      5
M-SC       XY24       839573.0      6
H-SC       XR         820997.0      7
X-MC       X40        559244.0      7
X-LC       X40        524975.0      6
X-MC       XY24       487647.0      3
           X40N       421715.0      4
X-SC       X40N       387950.0      4
M-SC       OX40N      341701.0      5
H-SC       OX40N      334293.0      4
X-SC       XY24       308992.0      3
H-SC       SR         296642.0      2
X-LC       X40N       241678.0      2
X-MC       XY25       226454.0      2
H-MC       XY24       213128.0      1
X-LC       XY24       210300.0      2
H-MC       AR         191029.0      2
M-SC       XR         182249.0      2
X-SC       X40        171128.0      2
M-LC       XR         131911.0      1
M-SC       AR         125099.0      2
X-LC       XY25       119670.0      2
L-SC       XR          95808.0      1
           OX40N       91820.0      1
H-LC       AR          71583.0      1
L-MC       XR          51455.0      1
M-MC       XY25        41749.0      1
L-LC       XY25        20581.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
